In [106]:
import pandas as pd
import numpy as np 
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [ ]:
my_data = pd.read_csv('anime/anime-dataset-2023.csv')

In [181]:
my_data.Aired.describe()

count    24901.000000
mean         0.841329
std          0.133795
min          0.000000
25%          0.796296
50%          0.879630
75%          0.925926
max          1.000000
Name: Aired, dtype: float64

In [10]:

#one-hot encode the genres
total_genres = list(set(", ".join(my_data["Genres"].unique()).split(", ")))
print(total_genres)
total_genres = ['Adventure', 'Drama', 'Gourmet', 'Romance', 'Sports', 'Action', 'Suspense', 'Fantasy', 'Horror', 'Hentai', 'Supernatural', 'Avant Garde', 'Erotica', 'Mystery', 'Ecchi', 'Sci-Fi', 'Girls Love', 'UNKNOWN', 'Award Winning', 'Slice of Life', 'Boys Love', 'Comedy']
my_data[["genre "+gg for gg in total_genres]] = 0
#print(my_data.columns)

def split_genre(row):
    gs = row["Genres"].split(", ")
    for gg in gs:
        row["genre "+gg] = 1
    return row

my_data = my_data.apply(split_genre, axis=1)
my_data

['UNKNOWN', 'Action', 'Sci-Fi', 'Drama', 'Fantasy', 'Slice of Life', 'Girls Love', 'Gourmet', 'Suspense', 'Horror', 'Adventure', 'Comedy', 'Romance', 'Award Winning', 'Supernatural', 'Mystery', 'Avant Garde', 'Erotica', 'Sports', 'Hentai', 'Boys Love', 'Ecchi']


,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,genre Erotica,genre Mystery,genre Ecchi,genre Sci-Fi,genre Girls Love,genre UNKNOWN,genre Award Winning,genre Slice of Life,genre Boys Love,genre Comedy
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,0,0,0,1,0,0,1,0,0,0
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,0,0,0,1,0,0,0,0,0,0
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,0,0,0,1,0,0,0,0,0,0
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,0,1,0,0,0,0,0,0,0,0
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24900,55731,Wu Nao Monu,UNKNOWN,无脑魔女,UNKNOWN,"Comedy, Fantasy, Slice of Life",No description available for this anime.,ONA,15.0,"Jul 4, 2023 to ?",...,0,0,0,0,0,0,0,1,0,1
24901,55732,Bu Xing Si: Yuan Qi,Blader Soul,捕星司·源起,UNKNOWN,"Action, Adventure, Fantasy",No description available for this anime.,ONA,18.0,"Jul 27, 2023 to ?",...,0,0,0,0,0,0,0,0,0,0
24902,55733,Di Yi Xulie,The First Order,第一序列,UNKNOWN,"Action, Adventure, Fantasy, Sci-Fi",No description available for this anime.,ONA,16.0,"Jul 19, 2023 to ?",...,0,0,0,1,0,0,0,0,0,0
24903,55734,Bokura no Saishuu Sensou,UNKNOWN,僕らの最終戦争,UNKNOWN,UNKNOWN,A music video for the song Bokura no Saishuu S...,Music,1.0,"Apr 23, 2022",...,0,0,0,0,0,1,0,0,0,0


In [28]:
my_data = my_data[~my_data['Name'].duplicated()]


In [33]:
from numpy import nan


def extract_year(date_str):
    date_str = str(date_str)
    if 'Unknown' in date_str:
        return nan  
    years = re.findall(r'\b(19\d{2}|20\d{2})\b', date_str)
    if len(years) == 2:  
        return (int(years[0]) + int(years[1])) // 2  
    elif years:
        return int(years[0])  
    else:
        return nan  # In case of a parsing error


my_data['Aired'] = my_data['Aired'].apply(extract_year)

In [40]:
my_data[my_data['Aired'].isna()]


,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,genre Erotica,genre Mystery,genre Ecchi,genre Sci-Fi,genre Girls Love,genre UNKNOWN,genre Award Winning,genre Slice of Life,genre Boys Love,genre Comedy
8263,20471,Aoki Uru,Uru in Blue,蒼きウル,UNKNOWN,Sci-Fi,"In March 1992, Gainax began the planning and p...",Movie,1.0,NaN,...,0,0,0,1,0,0,0,0,0,0
9445,26157,Shin-chan no Sanrinsha,Shin's Tricycle,しんちゃんのさんりんしゃ,UNKNOWN,UNKNOWN,The story about a real life 3 year old boy nam...,OVA,1.0,NaN,...,0,0,0,0,0,1,0,0,0,0
9466,26237,Cherry Blossom,UNKNOWN,チェリーブラッサム,UNKNOWN,Drama,An educational anime about what people should ...,OVA,1.0,NaN,...,0,0,0,0,0,0,0,0,0,0
9785,28581,Nintama Rantarou no Jishin Youjin Hi no Youjin,UNKNOWN,忍たま乱太郎の 地震用心 火の用心,UNKNOWN,Drama,An educational film about fire safety during e...,OVA,1.0,NaN,...,0,0,0,0,0,0,0,0,0,0
9786,28583,Nintama Rantarou no Shouboutai,UNKNOWN,忍たま乱太郎の消防隊,UNKNOWN,Drama,An educational film about fire brigades starri...,OVA,1.0,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24875,55701,Kimetsu no Yaiba: Hashira Geiko-hen,Demon Slayer: Kimetsu no Yaiba Hashira Trainin...,鬼滅の刃 柱稽古編,UNKNOWN,"Action, Fantasy",Hashira Training Arc of Kimetsu no Yaiba,TV,UNKNOWN,NaN,...,0,0,0,0,0,0,0,0,0,0
24876,55702,Kimetsu no Yaiba: Purātorēninguāku-hen,Demon slayer: Kimetsu no yaiba pillar training...,Kimetsu no Yaiba: Purātorēninguāku-hen,UNKNOWN,"Action, Comedy, Fantasy",Fourth season of Kimetsu no Yaiba,TV,UNKNOWN,NaN,...,0,0,0,0,0,0,0,0,0,1
24883,55710,Hana-Doll*: Reinterpretation of Flowering,UNKNOWN,華Doll* -Reinterpretation of Flowering-,UNKNOWN,UNKNOWN,"The ""Hana Ningyou Project"" (Flower Doll Projec...",UNKNOWN,UNKNOWN,NaN,...,0,0,0,0,0,1,0,0,0,0
24885,55716,Mechronicle,Mechronicle,Mechronicle,UNKNOWN,UNKNOWN,"Set in a mechanical world, Mechronicle tells t...",UNKNOWN,UNKNOWN,NaN,...,0,0,0,0,0,1,0,0,0,0


In [41]:
# add average year for unknown data
middle_year = my_data['Aired'].median()
my_data['Aired'].fillna(middle_year, inplace=True)
my_data['Aired'] = my_data['Aired'].astype(int)

# One Hot encode the Licensors

In [44]:
total_licenses = list(set(", ".join(my_data["Licensors"].unique()).split(", ")))
print(total_licenses)

#my_data[["licensor "+gg for gg in total_licenses]] = 0
#print(my_data.columns)
'''
def split_genre(row):
    gs = row["Genres"].split(", ")
    for gg in gs:
        row["genre "+gg] = 1
    return row

my_data = my_data.apply(split_genre, axis=1)
my_data
'''

['Dentsu Entertainment USA', 'ADV Films', 'Tokyopop', 'NYAV Post', 'Eleven Arts', 'Travel Compass', 'B.CMAY PICTURES', 'NBCUniversal Entertainment Japan', 'Cookie Jar Entertainment', 'Crunchyroll', 'Anime Midstream', 'Saban Entertainment', 'Synch-Point', '4Kids Entertainment', 'SoftCel Pictures', 'Directions', 'Kadokawa Pictures USA', 'Geneon Universal Entertainment', 'NuTech Digital', 'Marvel Entertainment', 'DreamWorks', 'Mill Creek Entertainment', 'DiC Entertainment', 'Hasbro', 'Enoki Films', 'Voyager Entertainment', 'Kadokawa', 'Pioneer LDC', 'Central Park Media', 'Geneon Entertainment USA', 'Hirameki International', 'Nelvana', 'Aniplex of America', 'AnimEigo', 'Sentai Filmworks', 'Adult Source Media', 'NIS America', 'Illumitoon Entertainment', 'Capcom', 'Harmony Gold', 'Bandai Visual USA', 'Ponycan USA', 'Miramax Films', 'Muse Communication', 'iQIYI', 'Shout! Factory', 'Ketchup Entertainment', 'JapanAnime', 'Ascendent Animation', 'Frontier Works', 'Arts Magic', 'Konami Cross Media

'\ndef split_genre(row):\n    gs = row["Genres"].split(", ")\n    for gg in gs:\n        row["genre "+gg] = 1\n    return row\n\nmy_data = my_data.apply(split_genre, axis=1)\nmy_data\n'

# One Hot encode the type

In [49]:
total_Types = list(set(", ".join(my_data["Type"].unique()).split(", ")))
print(total_Types)

my_data[["Type "+t for t in total_Types]] = 0
#print(my_data.columns)

def split_Type(row):
    gs = row["Type"].split(", ")
    for t in gs:
        row["Type "+t] = 1
    return row

my_data = my_data.apply(split_Type, axis=1)
my_da

['UNKNOWN', 'OVA', 'Special', 'ONA', 'Movie', 'Music', 'TV']


,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,genre Slice of Life,genre Boys Love,genre Comedy,Type UNKNOWN,Type OVA,Type Special,Type ONA,Type Movie,Type Music,Type TV
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,1998,...,0,0,0,0,0,0,0,0,0,1
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,2001,...,0,0,0,0,0,0,0,1,0,0
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,1998,...,0,0,0,0,0,0,0,0,0,1
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,2002,...,0,0,0,0,0,0,0,0,0,1
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,2004,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24900,55731,Wu Nao Monu,UNKNOWN,无脑魔女,UNKNOWN,"Comedy, Fantasy, Slice of Life",No description available for this anime.,ONA,15.0,2023,...,1,0,1,0,0,0,1,0,0,0
24901,55732,Bu Xing Si: Yuan Qi,Blader Soul,捕星司·源起,UNKNOWN,"Action, Adventure, Fantasy",No description available for this anime.,ONA,18.0,2023,...,0,0,0,0,0,0,1,0,0,0
24902,55733,Di Yi Xulie,The First Order,第一序列,UNKNOWN,"Action, Adventure, Fantasy, Sci-Fi",No description available for this anime.,ONA,16.0,2023,...,0,0,0,0,0,0,1,0,0,0
24903,55734,Bokura no Saishuu Sensou,UNKNOWN,僕らの最終戦争,UNKNOWN,UNKNOWN,A music video for the song Bokura no Saishuu S...,Music,1.0,2022,...,0,0,0,0,0,0,0,0,1,0


'OVA'

In [58]:
my_data['Episodes'].replace('UNKNOWN', np.nan, inplace=True)
my_data['Episodes'].fillna(my_data['Episodes'].median(), inplace=True)
my_data['Episodes'] = my_data['Episodes'].astype(float)
scaler = MinMaxScaler()
my_data['Aired'] = scaler.fit_transform(my_data[['Aired']])
my_data['Episodes'] = scaler.fit_transform(my_data[['Episodes']])


In [60]:
my_data

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,genre Slice of Life,genre Boys Love,genre Comedy,Type UNKNOWN,Type OVA,Type Special,Type ONA,Type Movie,Type Music,Type TV
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,0.008181,0.750000,...,0,0,0,0,0,0,0,0,0,1
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,0.000000,0.777778,...,0,0,0,0,0,0,0,1,0,0
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,0.008181,0.750000,...,0,0,0,0,0,0,0,0,0,1
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,0.008181,0.787037,...,0,0,0,0,0,0,0,0,0,1
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,0.016688,0.805556,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24900,55731,Wu Nao Monu,UNKNOWN,无脑魔女,UNKNOWN,"Comedy, Fantasy, Slice of Life",No description available for this anime.,ONA,0.004581,0.981481,...,1,0,1,0,0,0,1,0,0,0
24901,55732,Bu Xing Si: Yuan Qi,Blader Soul,捕星司·源起,UNKNOWN,"Action, Adventure, Fantasy",No description available for this anime.,ONA,0.005563,0.981481,...,0,0,0,0,0,0,1,0,0,0
24902,55733,Di Yi Xulie,The First Order,第一序列,UNKNOWN,"Action, Adventure, Fantasy, Sci-Fi",No description available for this anime.,ONA,0.004908,0.981481,...,0,0,0,0,0,0,1,0,0,0
24903,55734,Bokura no Saishuu Sensou,UNKNOWN,僕らの最終戦争,UNKNOWN,UNKNOWN,A music video for the song Bokura no Saishuu S...,Music,0.000000,0.972222,...,0,0,0,0,0,0,0,0,1,0


In [63]:
features = my_data.drop(columns=[
    'Name', 'English name', 'Other name', 'Score', 'Genres', 'Synopsis', 'Type', 
    'Premiered', 'Status', 'Producers', 'Licensors', 'Studios', 'Source', 
    'Duration', 'Rating', 'Rank', 'Popularity', 'Favorites', 'Scored By', 
    'Members', 'Image URL'
])

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(features)

# Create a DataFrame for the similarity matrix, setting the index and columns to help identify the rows and columns easily
cosine_sim_df = pd.DataFrame(cosine_sim, index=my_data['Name'], columns=my_data['Name'])



In [65]:
def get_recommendations(anime_name, cosine_sim=cosine_sim, df=my_data, top_n=5):
    # Get the index of the anime that matches the name
    idx = df.index[df['Name'] == anime_name].tolist()[0]

    # Get the pairwsie similarity scores of all animes with that anime
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the animes based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the most similar animes
    sim_scores = sim_scores[1:top_n+1]  # Skip the first one because it is the query anime itself

    # Get the anime indices
    anime_indices = [i[0] for i in sim_scores]

    # Return the top most similar animes
    return df['Name'].iloc[anime_indices]


In [86]:
features

,anime_id,Episodes,Aired,genre Adventure,genre Drama,genre Gourmet,genre Romance,genre Sports,genre Action,genre Suspense,...,genre Slice of Life,genre Boys Love,genre Comedy,Type UNKNOWN,Type OVA,Type Special,Type ONA,Type Movie,Type Music,Type TV
0,1,0.008181,0.750000,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,5,0.000000,0.777778,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,6,0.008181,0.750000,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,7,0.008181,0.787037,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,8,0.016688,0.805556,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24900,55731,0.004581,0.981481,0,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,0,0
24901,55732,0.005563,0.981481,1,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
24902,55733,0.004908,0.981481,1,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
24903,55734,0.000000,0.972222,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [99]:
vectorizer = TfidfVectorizer(stop_words='english')

# Documents
documents = [
    "Cat is on the table.",
    "Dog is under the table.",
    "Cat and dog are friends."
]

# Application de fit_transform
tfidf_matrix = vectorizer.fit_transform(documents)

# Affichage du résultat sous forme d'array
print(tfidf_matrix.toarray())

# Affichage du vocabulaire appris
print(vectorizer.get_feature_names_out())

[[0.70710678 0.         0.         0.70710678]
 [0.         0.70710678 0.         0.70710678]
 [0.51785612 0.51785612 0.68091856 0.        ]]
['cat' 'dog' 'friends' 'table']


# Recommandations using the synopsis

In [103]:
my_data_syn = my_data.loc[:  , ['anime_id' , 'Name' , 'Synopsis']]
my_data_syn

,anime_id,Name,Synopsis
0,1,Cowboy Bebop,"Crime is timeless. By the year 2071, humanity ..."
1,5,Cowboy Bebop: Tengoku no Tobira,"Another day, another bounty—such is the life o..."
2,6,Trigun,"Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,Robin Sena is a powerful craft user drafted in...
4,8,Bouken Ou Beet,It is the dark century and the people are suff...
...,...,...,...
24900,55731,Wu Nao Monu,No description available for this anime.
24901,55732,Bu Xing Si: Yuan Qi,No description available for this anime.
24902,55733,Di Yi Xulie,No description available for this anime.
24903,55734,Bokura no Saishuu Sensou,A music video for the song Bokura no Saishuu S...


In [104]:
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
my_data_syn['Synopsis'] = my_data_syn['Synopsis'].fillna('')
my_data_syn['Synopsis'] = my_data_syn['Synopsis'].str.lower()
my_data_syn['Synopsis'] = my_data_syn['Synopsis'].str.replace(r'[^\w\s]+', '')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(my_data_syn['Synopsis'])

#Output the shape of tfidf_matrix
tfidf_matrix.shap

(24901, 51088)

In [177]:
tfidf_matrix

<24901x51088 sparse matrix of type '<class 'numpy.float64'>'
	with 639217 stored elements in Compressed Sparse Row format>

In [107]:
# Compute the cosine similarity matrix
cosine_synopsis = linear_kernel(tfidf_matrix, tfidf_matrix)

In [158]:
indices = pd.Series(my_data_syn.index, index=my_data_syn['Name']).drop_duplicates()

In [159]:
indices

Name
Cowboy Bebop                           0
Cowboy Bebop: Tengoku no Tobira        1
Trigun                                 2
Witch Hunter Robin                     3
Bouken Ou Beet                         4
                                   ...  
Wu Nao Monu                        24900
Bu Xing Si: Yuan Qi                24901
Di Yi Xulie                        24902
Bokura no Saishuu Sensou           24903
Shijuuku Nichi                     24904
Length: 24901, dtype: int64

In [175]:
def get_recommendations(title, cosine_sim=cosine_synopsis, suggest_amount=15):
    try:
        idx = indices[title]
        print(idx)
    except KeyError:
        raise ValueError("Anime  not found in matrix.")


    return get_recommendations_by_id(idx,cosine_sim, suggest_amount)


def get_recommendations_by_id(idx, cosine_sim=cosine_synopsis, suggest_amount=15) : 
    sim_scores = list(enumerate(cosine_sim[idx])) 
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    max_amount = len(sim_scores)
    if suggest_amount > max_amount:
        suggest_amount = max_amount
    
    sim_scores = sim_scores[1:suggest_amount]
    
    # Get the anime indices
    anime_indices = [i[0] for i in sim_scores]
    
    # Map indices to anime_id
    anime_ids = my_data_syn.iloc[anime_indices]['anime_id'].values
    
    # Calculate the normalized scores (L2 normalization)
    scores = np.array([i[1] for i in sim_scores])
    norm_scores = scores / np.linalg.norm(scores)
    
    ret_df = pd.DataFrame({
        'Anime' : my_data_syn.iloc[anime_indices]['Name'] , 
        'Score' : norm_scores , 
        'anime_id': anime_ids , 
        'Position' : range(1 ,  suggest_amount)
    }
    )
    return ret_df


In [180]:
get_recommendations('One Piece')

11


,Anime,Score,anime_id,Position
13409,One Piece: Episode of East Blue - Luffy to 4-n...,0.419231,36215,1
4038,One Piece: Romance Dawn Story,0.328381,5252,2
5494,One Piece Film: Strong World Episode 0,0.324337,8740,3
14699,One Piece Movie 14: Stampede,0.304782,38234,4
6823,One Piece Film: Z,0.266921,12859,5
1128,One Piece: Oounabara ni Hirake! Dekkai Dekkai ...,0.263736,1237,6
3514,One Piece Film: Strong World,0.261924,4155,7
9266,One Piece 3D2Y: Ace no shi wo Koete! Luffy Nak...,0.244479,25161,8
8065,Kaizoku Ouji,0.235319,19505,9
21804,One Piece Film: Red,0.216839,50410,10
